<a href="https://colab.research.google.com/github/GokayToga/ThermalNet/blob/main/ThermalNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>